In [2]:
# Dependencies
import jsonimport pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1]:
# Mike's code
df_path = "Resources/videos.csv"
df = pd.read_csv(df_path)

df[["publish date", "asdf"]] = df["publish_time"].str.split(pat="T", n=1, expand=True)


df[["publish time", "trash"]] = df["asdf"].str.split(pat=".", n=1, expand=True)


df["publish date"] = pd.to_datetime(df["publish date"])
df["publish_date"] = df["publish date"].dt.strftime('%m/%d/%Y')
df.head()

In [ ]:
# Riley's code

In [31]:
category_json_path = "Resources/US_category_id.json"
category_dict = {'category_id': [], 'Category': []}
indexer = 0

with open(category_json_path) as json_data:
  data = json.load(json_data)

items = data['items']
items

for item in items:
    
    category_dict['category_id'].append(data['items'][indexer]['id'])
    category_dict['Category'].append(data['items'][indexer]['snippet']['title'])
    indexer = indexer + 1
        
category_dict

{'category_id': ['1',
  '2',
  '10',
  '15',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44'],
 'Category': ['Film & Animation',
  'Autos & Vehicles',
  'Music',
  'Pets & Animals',
  'Sports',
  'Short Movies',
  'Travel & Events',
  'Gaming',
  'Videoblogging',
  'People & Blogs',
  'Comedy',
  'Entertainment',
  'News & Politics',
  'Howto & Style',
  'Education',
  'Science & Technology',
  'Nonprofits & Activism',
  'Movies',
  'Anime/Animation',
  'Action/Adventure',
  'Classics',
  'Comedy',
  'Documentary',
  'Drama',
  'Family',
  'Foreign',
  'Horror',
  'Sci-Fi/Fantasy',
  'Thriller',
  'Shorts',
  'Shows',
  'Trailers']}

In [32]:
category_df = pd.DataFrame(category_dict)
df = df.merge(category_df, how='inner', on='category_id')

NameError: name 'df' is not defined

In [ ]:
# Jackie's code

In [ ]:
print(nothing)

In [ ]:
# Brian's code

In [ ]:
#this is my test cell to merge